# Pauli Transfer Matrix for the Qubit Device

## Import packages

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [12]:
import numpy as np
from qutip import *
from qutip.qip.device import *
from qutip.qip.operations import *
from qutip.qip.circuit import QubitCircuit
from qaoa_with_cat_qubits import *

Create a function that return the PTM

In [23]:
def ideal_ptm(U):
    """ Returns the Pauli Transfer Matrix of a unitary operator U """
    paulis = [identity(2), sigmax(), sigmay(), sigmaz()]
    M = np.zeros((d**2, d**2), dtype=complex)
    
    for j in range(d**2):
        Lambda = U * paulis[j] * U.dag()
        for i in range(d**2):
            M[i,j] = 1/d * ((paulis[i]*Lambda).tr()).real
    return M

## RX-gate

In [24]:
d = 2  # dimension
tau = 10  # gate time
# Load the average gate fidelity for CV device
file = np.load('../../data/average_gate_fidelity/cv_avg_fid_rx.npz')
f_bar = np.mean(file['avg'])
# Find the corresponding T1
gamma = 2 * (d + 1) / (d * tau) * (1 - f_bar)
T1 = 1 / (gamma)

# Qubit Processor
qp = QubitProcessor(N=1, T1=T1)
qp_inv = QubitProcessor(N=1, T1=None)

# Pauli matrices
sigma = [qeye(2), sigmax(), sigmay(), sigmaz()]

arg = np.pi/2

# Pauli transfer matrix
R = np.zeros((d**2, d**2))
R_inv = np.zeros((d**2, d**2))

# Quantum circuit
qc = QubitCircuit(1)
qc.add_gate("RX", 0, None, arg)

# Target
U = (-1j*sigmax()*arg/2).expm()

# Create PTM
for j in range(d**2):
    result = qp.run_state(init_state=sigma[j], qc=qc)
    result_inv = qp_inv.run_state(init_state=sigma[j], qc=qc)
    Lambda = result.states[-1]
    Lambda_inv = result_inv.states[-1]
    for i in range(d**2):
        R[i,j] = 1/d * ((sigma[i]*Lambda).tr()).real
        R_inv[i,j] = 1/d * ((sigma[i]*Lambda_inv).tr()).real

In [25]:
error_channel = np.around(R @ np.linalg.inv(R_inv), 3)
Qobj(error_channel)

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 1.     0.     0.     0.   ]
 [ 0.     0.979  0.     0.   ]
 [-0.026  0.     0.969  0.007]
 [ 0.026  0.     0.006  0.969]]

In [26]:
A = np.zeros((4,4,4,4),dtype='complex')
P = [qeye(2),sigmax(),sigmay(),sigmaz()]
for i in range(4):
    for j in range(4):
        for m in range(4):
            for n in range(4):
                A[i,j,m,n] = (P[i]*P[m]*P[j]*P[n]).tr()
A = A.reshape((16,16))
A_inv = np.linalg.inv(A)

In [28]:
chi = d*Qobj((A_inv @ error_channel.reshape((4**2,1))).reshape((4,4)),dims=[[[2],[2]],[[2],[2]]],superrep='chi')
chi

Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = (4, 4), type = super, isherm = True, superrep = chi
Qobj data =
[[ 0.97925+0.j       0.     -0.00025j -0.0065 +0.j       0.0065 +0.j     ]
 [ 0.     +0.00025j  0.01025+0.j       0.     -0.0065j   0.     -0.0065j ]
 [-0.0065 +0.j       0.     +0.0065j   0.00525+0.j       0.00325+0.j     ]
 [ 0.0065 +0.j       0.     +0.0065j   0.00325+0.j       0.00525+0.j     ]]

## RZ-gate

In [24]:
d = 2  # dimension
tau = 2  # gate time
# Load the average gate fidelity
file = np.load('../../data/average_gate_fidelity/cv_avg_fid_rz.npz')
f_bar = np.mean(file['avg'])
# Find the corresponding T1
gamma = 2 * (d + 1) / (d * tau) * (1 - f_bar)
T1 = 1 / (gamma)

# Qubit Processor
qp = QubitProcessor(N=1, T1=T1)
qp_inv = QubitProcessor(N=1, T1=None)

# Pauli matrices
sigma = [qeye(2), sigmax(), sigmay(), sigmaz()]

arg = np.pi/2

# Pauli transfer matrix
R = np.zeros((d**2, d**2))
R_inv = np.zeros((d**2, d**2))

# Quantum circuit
qc = QubitCircuit(1)
qc.add_gate("RZ", 0, None, arg)

# Target
U = (-1j*sigmaz()*arg/2).expm()

# Create PTM
for j in range(d**2):
    result = qp.run_state(init_state=sigma[j], qc=qc)
    result_inv = qp_inv.run_state(init_state=sigma[j], qc=qc)
    Lambda = result.states[-1]
    Lambda_inv = result_inv.states[-1]
    for i in range(d**2):
        R[i,j] = 1/d * ((sigma[i]*Lambda).tr()).real
        R_inv[i,j] = 1/d * ((sigma[i]*Lambda_inv).tr()).real

In [25]:
error_channel = np.around(R @ np.linalg.inv(R_inv), 3)
Qobj(error_channel)

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[1.    0.    0.    0.   ]
 [0.    0.995 0.    0.   ]
 [0.    0.    0.995 0.   ]
 [0.011 0.    0.    0.989]]

In [26]:
chi = d*Qobj((A_inv @ error_channel.reshape((4**2,1))).reshape((4,4)),dims=[[[2],[2]],[[2],[2]]],superrep='chi')
chi

Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = (4, 4), type = super, isherm = True, superrep = chi
Qobj data =
[[ 9.9475e-01+0.j       0.0000e+00+0.j       0.0000e+00+0.j
   2.7500e-03+0.j     ]
 [ 0.0000e+00+0.j       2.7500e-03+0.j       0.0000e+00-0.00275j
   0.0000e+00+0.j     ]
 [ 0.0000e+00+0.j       0.0000e+00+0.00275j  2.7500e-03+0.j
   0.0000e+00+0.j     ]
 [ 2.7500e-03+0.j       0.0000e+00+0.j       0.0000e+00+0.j
  -2.5000e-04+0.j     ]]

## RZZ-gate

In [ ]:
def product(*args, repeat=1):
    # product('ABCD', 'xy') --> Ax Ay Bx By Cx Cy Dx Dy
    # product(range(2), repeat=3) --> 000 001 010 011 100 101 110 111
    pools = [tuple(pool) for pool in args] * repeat
    result = [[]]
    for pool in pools:
        result = [x + [y] for x in result for y in pool]
    for prod in result:
        yield list(prod)

In [ ]:
d = 4  # dimension
tau = 2  # gate time
# Load the average gate fidelity
file = np.load('../../data/average_gate_fidelity/cv_avg_fid_rzz.npz')
f_bar = np.mean(file['avg'])
# Find the corresponding T1
gamma = (d + 1) / (d * tau) * (1 - f_bar)
T1 = 1 / (gamma)

# Qubit Processor
qp = QubitProcessor(N=1, T1=T1)
qp_inv = QubitProcessor(N=1, T1=None)

# Create all tensor products of pauli matrices
sigma = list(map(tensor, product(P, repeat=2)))

arg = np.pi/2

# Pauli transfer matrix
R = np.zeros((d**2, d**2))
R_inv = np.zeros((d**2, d**2))

# Quantum circuit
qc = QubitCircuit(1)
qc.add_gate("RZ", 0, None, arg)

# Target
U = (-1j*sigmaz()*arg/2).expm()

# Create PTM
for j in range(d**2):
    result = qp.run_state(init_state=sigma[j], qc=qc)
    result_inv = qp_inv.run_state(init_state=sigma[j], qc=qc)
    Lambda = result.states[-1]
    Lambda_inv = result_inv.states[-1]
    for i in range(d**2):
        R[i,j] = 1/d * ((sigma[i]*Lambda).tr()).real
        R_inv[i,j] = 1/d * ((sigma[i]*Lambda_inv).tr()).real